In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.io import fits as fits
import healpy as hp
import pysm 
from pysm.nominal import models
from pysm.common import convert_units
import camb
from camb import model, initialpower

from numpy import linalg as LA

import utils

from utils import bin_l as bin_l
plt.rcParams['figure.figsize'] = (10.0, 8.0)

NSIDE = 1024
ORDERING = RING in fits file
INDXSCHM = IMPLICIT


# Public area

In [41]:
ali_ma = hp.read_map('/smc/jianyao/Ali_maps/ali_mask_wo_edge.fits', verbose=False)

# I Q U; uK_CMB, 2048
dust_95 = hp.ud_grade(hp.read_map('/fnx/jianyao/1205_skyinband_AliCPT/95GHz/thermaldust_map_95GHz.fits', field = None, verbose = False), nside_out=1024)
dust_150 = hp.ud_grade(hp.read_map('/fnx/jianyao/1205_skyinband_AliCPT/150GHz/thermaldust_map_150GHz.fits', field = None, verbose = False), nside_out=1024)

# IQU, K_CMB, 2048
dust_353 = 1e6*hp.ud_grade(hp.read_map('/fnx/jianyao/1205_skyinband_HFI_353/detector_F353/thermaldust_map_detector_F353.fits', field = None, verbose = False), nside_out = 1024)

sync_95 = hp.ud_grade(hp.read_map('/fnx/jianyao/1205_skyinband_AliCPT/95GHz/synchrotron_map_95GHz.fits',field = None, verbose = False), nside_out=1024)
sync_150 = hp.ud_grade(hp.read_map('/fnx/jianyao/1205_skyinband_AliCPT/150GHz/synchrotron_map_150GHz.fits',field = None, verbose = False), nside_out=1024)
sync_353= 1e6*hp.ud_grade(hp.read_map('/fnx/jianyao/1205_skyinband_HFI_353/detector_F353/synchrotron_map_detector_F353.fits',field = None, verbose = False), nside_out=1024)

#uK_CMB, 2048, IQU
cmb = hp.ud_grade(hp.read_map('/fnx/jianyao/1205_skyinband_AliCPT/95GHz/cmb_map_95GHz.fits', field = None, verbose = False), nside_out=1024)

In [2]:
nside = 1024; L = 2000; Q = 200

In [ ]:
cmb_ma = utils.Mask(cmb) 
cmb_cls = hp.anafast(cmb_ma, lmax = lmax, nspec = 3)
ell = np.arange(len(cmb_cls[0])); Q = 200
Ell = utils.get_ell(lmax, Q); Ell2 = utils.l2(Ell)

clee_clean = utils.bin_l(cmb_cls[1], L, Q); clbb_clean = utils.bin_l(cmb_cls[2], L, Q)